This code creates binary files containing test TCP messages with directives/requests/TM transfer frames containing CLCWs for testing the COP implementation in GNURadio  
Run all the cells under 'Run this' which contains helper classes/functions   
Run the specific cell under 'Optional' according to the intended directive/request/CLCWs to generate  
Change the parameters under each cell eg retransmit_flag for clcw, optional_paramter for Set V(S) directive to generate different versions for testing  
Change the endian parameter in the first code cell to change the endianess of the TCP messages

# Run this

In [35]:
#Parameters
endian = 'big' #'big'/'little'

TcTransferFrame

In [9]:
import struct
import enum
import copy


# This class is used to decode and manipulate a TC transfer frame, compliant to CCSDS 232.0-B-3. It includes also support
# for the security protocol, as defined by the same standard.
class TcTransferFrame:

    # The type of TC frame.
    class FrameType(enum.Enum):
        AD = 0 # Sequence-controlled: bypass flag set to 0, control command flag set to 0
        RESERVED = 1 # Reserved for future use: bypass flag set to 0, control command flag set to 1
        BD = 2 # Expedited: bypass flag set to 1, control command flag set to 0
        BC = 3 # Control command (SetVR, Unlock): bypass flag set to 1, control command flag set to 1

    # The type of control command.
    class ControlCommandType(enum.Enum):
        UNLOCK = 0 # COP-1 Unlock directive
        SET_VR = 1 # COP-1 SetV(R) directive
        RESERVED = 2 # Reserved for future use

    # The meaning of the sequence flag in case of TC segment service.
    class SequenceFlagType(enum.Enum):
        FIRST = 0 # First block
        CONTINUE = 1 # Continuation block
        LAST = 2 # Last block
        NO_SEGMENT = 3 # No segmentation
        
    # Length of the TC Frame header
    TC_PRIMARY_HEADER_LENGTH = 5
    # Maximum length of the TC Frame according to CCSDS
    MAX_TC_FRAME_LENGTH = 1024

    def __init__(self, frame, segmented, fecf_present, sec_header_length=0, sec_trailer_length=0, req_tag=0, operation_tag=0):
        self.frame = frame # The decoded transfer frame.
        self.segmented = segmented
        self.fecf_present = fecf_present # Frame error control field presence flag.
        self.security_header_length = sec_header_length
        self.security_trailer_length = sec_trailer_length
        self.req_tag = req_tag
        self.operation_tag = operation_tag

        # Check transfer frame version number
        two_octets = struct.unpack('>H', frame[:2])[0]
        tfvn = (two_octets & 0xC000) >> 14
        if tfvn != 0:
            raise ValueError(f"Transfer Frame Version Number: expected 0, actual {tfvn}")

        # Primary header fields
        self.transfer_frame_version_number = tfvn # Transfer frame version number.
        self.bypass_flag = (two_octets & 0x2000) != 0
        self.control_command_flag = (two_octets & 0x1000) != 0
        self.spacecraft_id = two_octets & 0x03FF # Spacecraft id.

        two_octets = struct.unpack('>H', frame[2:4])[0] #next 2 octets
        self.virtual_channel_id = (two_octets & 0xFC00) >> 10 # Virtual channel id.
        self.frame_length = (two_octets & 0x03FF) + 1
        self.control_command_type = None
        self.set_vr_value = None
        self.ocf_present = None

        if self.frame_length != len(frame):
            raise ValueError(f"Wrong Frame Length: expected {len(frame)}, actual {self.frame_length}")

        self.virtual_channel_frame_count = struct.unpack('B', frame[4:5])[0] #last octet # Frame count on the virtual channel, this frame belongs to.

        if self.security_header_length > 0:
            self.data_field_start = self.TC_PRIMARY_HEADER_LENGTH + (1 if self.segmented else 0) + self.security_header_length # Data field start offset from the beginning of the frame.
            self.data_field_length = self.frame_length - self.data_field_start - self.security_trailer_length - (2 if self.fecf_present else 0) # Length of the data field (i.e. excluding FECF, OCF and security fields, if present
        else:
            self.data_field_start = self.TC_PRIMARY_HEADER_LENGTH
            self.data_field_length = self.frame_length - self.data_field_start - (2 if self.fecf_present else 0)

        if self.get_frame_type() == TcTransferFrame.FrameType.BC:
            data_field_length = len(frame) - self.data_field_start - (2 if self.fecf_present else 0) - self.security_trailer_length
            if data_field_length == 1:
                self.control_command_type = TcTransferFrame.ControlCommandType.UNLOCK if frame[self.data_field_start + self.security_header_length] == 0x00 else TcTransferFrame.ControlCommandType.RESERVED
            elif data_field_length == 3:
                if frame[self.data_field_start] == 0x82 and frame[self.data_field_start + 1] == 0x00:
                    self.control_command_type = TcTransferFrame.ControlCommandType.SET_VR
                    self.set_vr_value = struct.unpack('B', frame[self.data_field_start + 2:self.data_field_start + 3])[0]
                else:
                    self.control_command_type = TcTransferFrame.ControlCommandType.RESERVED
            else:
                self.control_command_type = TcTransferFrame.ControlCommandType.RESERVED

        if self.segmented and self.get_frame_type() != TcTransferFrame.FrameType.BC:
            seg_header = frame[self.TC_PRIMARY_HEADER_LENGTH]
            self.map_id = seg_header & 0x3F
            self.sequence_flag = TcTransferFrame.SequenceFlagType((seg_header & 0xC0) >> 6)

        if self.fecf_present:
            self.valid = self.check_validity() # FECF result: valid value only if fecfPresent is true. If there is no FECF, then the frame is considered valid.
        else:
            self.valid = True

    # Check fecf
    def check_validity(self):
        crc16 = self.get_crc16(self.frame,0,self.frame_length-2)
        crc_from_frame = self.get_fecf()
        return crc16 == crc_from_frame

    # Return frame type (BC/BD/AD)
    def get_frame_type(self):
        if self.bypass_flag:
            return TcTransferFrame.FrameType.BC if self.control_command_flag else TcTransferFrame.FrameType.BD
        else:
            return TcTransferFrame.FrameType.RESERVED if self.control_command_flag else TcTransferFrame.FrameType.AD

    def is_idle_frame(self):
        return False

    # Return bypass flag
    def is_bypass_flag(self):
        return self.bypass_flag

    # Return control command flag
    def is_control_command_flag(self):
        return self.control_command_flag

    # Return control command type (unlock/set v(r))
    def get_control_command_type(self):
        return self.control_command_type

    # Return set v(r) value
    def get_set_vr_value(self):
        return self.set_vr_value

    # Return map id
    def get_map_id(self):
        return self.map_id

    # Return sequence flag (first/continue/last/no segment)
    def get_sequence_flag(self):
        return self.sequence_flag

    # Return segmented
    def is_segmented(self):
        return self.segmented

    # Return security
    def is_security_used(self):
        return self.security_header_length != 0 or self.security_trailer_length != 0

    # Return security header length
    def get_security_header_length(self):
        return self.security_header_length

    # Return security trailer length
    def get_security_trailer_length(self):
        return self.security_trailer_length

    # Return a copy of security header
    def get_security_header_copy(self):
        return copy.deepcopy(self.frame[self.TC_PRIMARY_HEADER_LENGTH + (1 if self.segmented else 0):self.TC_PRIMARY_HEADER_LENGTH + (1 if self.segmented else 0) + self.security_header_length])

    # Return a copy of security trailer
    def get_security_trailer_copy(self):
        return copy.deepcopy(self.frame[-(2 if self.fecf_present else 0) - self.security_trailer_length:-2 if self.fecf_present else None])

    # Calculate crc16 
    def get_crc16(self, frame, offset, length) -> int:
        crc = 0xFFFF
        for i in range(offset, offset + length):
            crc ^= frame[i] << 8
            for _ in range(8):
                if crc & 0x8000:
                    crc = (crc << 1) ^ 0x1021
                else:
                    crc <<= 1
        return crc & 0xFFFF

    # Return fecf
    def get_fecf(self):
        return struct.unpack('>H', self.frame[-2:])[0]

    # Return transfer frame length
    def get_length(self):
        return len(self.frame)

    # Return fecf present
    def is_fecf_present(self):
        return self.fecf_present

    # Return transfer frame version number
    def get_transfer_frame_version_number(self):
        return self.transfer_frame_version_number

    # Return spacecraft id
    def get_spacecraft_id(self):
        return self.spacecraft_id

    # Return virtual channel id
    def get_virtual_channel_id(self):
        return self.virtual_channel_id

    # Return virtual channel frame count
    def get_virtual_channel_frame_count(self):
        return self.virtual_channel_frame_count

    # Return start index of data field 
    def get_data_field_start(self):
        return self.data_field_start

    # Return valid
    def is_valid(self):
        return self.valid

    # Return idle frame
    def is_idle_frame(self):
        return False

    # Return a copy of the frame data field.
    def get_data_field_copy(self):
        return copy.deepcopy(self.frame[self.data_field_start:self.data_field_start + self.get_data_field_length()])

    # Return the length of the data field.
    def get_data_field_length(self):
        return self.data_field_length
    
    # Return request tag
    def get_req_tag(self):
        return self.req_tag
    
    # Return operation tag
    def get_operation_tag(self):
        return self.operation_tag

    def __str__(self):
        return (
            f"TcTransferFrame{{bypassFlag={self.bypass_flag}, controlCommandFlag={self.control_command_flag}, "
            f"frameLength={self.frame_length}, segmented={self.segmented}, mapId={self.map_id}, "
            f"sequenceFlag={self.sequence_flag}, controlCommandType={self.control_command_type}, "
            f"setVrValue={self.set_vr_value}, securityHeaderLength={self.security_header_length}, "
            f"securityTrailerLength={self.security_trailer_length}, fecfPresent={self.fecf_present}, "
            f"ocfPresent={self.ocf_present}, transferFrameVersionNumber={self.transfer_frame_version_number}, "
            f"spacecraftId={self.spacecraft_id}, virtualChannelId={self.virtual_channel_id}, "
            f"virtualChannelFrameCount={self.virtual_channel_frame_count}, valid={self.valid}}}"
        )


TcTransferFrameBuilder

In [10]:
import struct
import numpy as np

# Create a TcTransferFrame object from a valid parameters indicating the fields of TC transfer frame header and data
class TcTransferFrameBuilder:
    def __init__(self, fecf_present):
        self.fecf_present = fecf_present
        self.bypass_flag = False
        self.control_command_flag = False
        self.spacecraft_id = 0
        self.virtual_channel_id = 0
        self.frame_sequence_number = 0
        self.free_user_data_length = 0
        self.map_id = 0
        self.segmented = False
        self.sequence_flag = 0
        self.security_header = bytearray()
        self.security_trailer = bytearray()
        self.payload_units = []
        self.operation_tag = 0
        self.req_tag = 0

        self.MAX_TC_FRAME_LENGTH = 1024
        self.TC_PRIMARY_HEADER_LENGTH = 5

        # Calculate initial free user data length
        self.free_user_data_length = self.compute_max_user_data_length()

    # Calculate the max possible user data length
    def compute_max_user_data_length(self):
        return self.MAX_TC_FRAME_LENGTH - self.TC_PRIMARY_HEADER_LENGTH - (2 if self.fecf_present else 0)

    # Set bypass flag
    def set_bypass_flag(self, bypass_flag):
        self.bypass_flag = bypass_flag
        return self

    # Set control command flag
    def set_control_command_flag(self, control_command_flag):
        self.control_command_flag = control_command_flag
        return self

    # Set spacecraft id
    def set_spacecraft_id(self, spacecraft_id):
        if not (0 <= spacecraft_id <= 1023):
            raise ValueError("Spacecraft ID must be 0 <= SC ID <= 1023, actual {}".format(spacecraft_id))
        self.spacecraft_id = spacecraft_id
        return self

    # Set virtual channel id
    def set_virtual_channel_id(self, virtual_channel_id):
        if not (0 <= virtual_channel_id <= 63):
            raise ValueError("Virtual Channel ID must be 0 <= VC ID <= 63, actual {}".format(virtual_channel_id))
        self.virtual_channel_id = virtual_channel_id
        return self

    # Set frame sequence number
    def set_frame_sequence_number(self, frame_sequence_number):
        if not (0 <= frame_sequence_number <= 255):
            raise ValueError("Frame Sequence Number must be 0 <= seq. num. <= 255, actual {}".format(frame_sequence_number))
        self.frame_sequence_number = frame_sequence_number
        return self

    # Set segmented flag, sequence flag, map id
    def set_segment(self, sequence_flag, map_id):
        if not (0 <= map_id <= 63):
            raise ValueError("Map ID must be 0 <= Map ID <= 64, actual {}".format(map_id))
        if self.is_full():
            raise ValueError("TC Frame already full")
        if not self.segmented:
            self.segmented = True
            self.free_user_data_length -= 1
        self.sequence_flag = sequence_flag
        self.map_id = map_id
        return self

    # Set unlock control command (for BC unlock frame)
    def set_unlock_control_command(self):
        # CCSDS 232.0-B-3, 4.1.3.3.2
        self.add_data(bytes([0x00]))
        return self

    # Set set v(r) control command (for BC set v(r) frame)
    def set_set_vr_control_command(self, frame_sequence_number):
        if not (0 <= frame_sequence_number <= 255):
            raise ValueError("Set V(R) Sequence Number must be 0 <= V(R) <= 255, actual {}".format(frame_sequence_number))
        self.add_data(bytes([0x82, 0x00, frame_sequence_number]))
        return self

    # Set security header and trailer
    def set_security(self, header, trailer):
        if header is None and trailer is None:
            return self
        if self.is_full():
            raise ValueError("TC Frame already full")
        security_data_size = (len(header) if header else 0) + (len(trailer) if trailer else 0)
        if self.free_user_data_length < security_data_size:
            raise ValueError("TC Frame cannot accommodate additional {} bytes, remaining space is {} bytes".format(security_data_size, self.free_user_data_length))
        self.security_header = header if header else bytearray()
        self.security_trailer = trailer if trailer else bytearray()
        self.free_user_data_length -= security_data_size
        return self
    
    # Set request tag
    def set_req_tag(self, req_tag):
        self.req_tag = req_tag
        return self
    
    # Set operation tag
    def set_operation_tag(self, operation_tag):
        self.operation_tag = operation_tag
        return self

    # Add user data
    def add_data(self, data):
        # Compute if you can add the requested amount
        data_to_be_written = min(self.free_user_data_length, len(data))
        not_written_data = len(data) - data_to_be_written
        if data_to_be_written > 0:
            self.payload_units.append(data[:data_to_be_written])
            self.free_user_data_length -= data_to_be_written
        return not_written_data

    # Get user data length
    def get_free_user_data_length(self):
        return self.free_user_data_length

    # Get whether user data field is full
    def is_full(self):
        return self.free_user_data_length == 0
    
    # Calculate crc16
    def get_crc16(self, data: bytes, start: int, length: int) -> int:
        crc = 0xFFFF
        for i in range(start, start + length):
            crc ^= data[i] << 8
            for _ in range(8):
                if crc & 0x8000:
                    crc = (crc << 1) ^ 0x1021
                else:
                    crc <<= 1
        return crc & 0xFFFF

    # Build and return the tc transfer frame
    def build(self):
        payload_data_length = sum(len(pu) for pu in self.payload_units)
        frame_length = self.TC_PRIMARY_HEADER_LENGTH + payload_data_length + (2 if self.fecf_present else 0) + (1 if self.segmented else 0)
        if self.security_header:
            frame_length += len(self.security_header)
        if self.security_trailer:
            frame_length += len(self.security_trailer)

        encoded_frame = bytearray()
        encoded_frame.extend(struct.pack('>H', (self.bypass_flag << 13) | (self.control_command_flag << 12) | self.spacecraft_id))
        encoded_frame.extend(struct.pack('>H', (self.virtual_channel_id << 10) | (frame_length - 1)))
        encoded_frame.append(self.frame_sequence_number)

        if self.segmented:
            to_write = (self.sequence_flag.value << 6) | self.map_id
            encoded_frame.append(to_write)

        # If security header, write it
        encoded_frame.extend(self.security_header)

        # Write the user data
        for pu in self.payload_units:
            encoded_frame.extend(pu)

        # If security trailer, write it
        encoded_frame.extend(self.security_trailer)

        # Compute and write the FECF (if present, 2 bytes)
        if self.fecf_present:
            crc = np.uint16(0)
            crc = self.get_crc16(encoded_frame, 0, len(encoded_frame) - 2)
            encoded_frame.extend(struct.pack('>H', crc))

        # Return the frame
        return TcTransferFrame(bytes(encoded_frame), lambda vc: self.segmented, self.fecf_present, len(self.security_header), len(self.security_trailer), self.req_tag, self.operation_tag)


BuildRandomTcTransferFrame

In [11]:
# Build a TC transfer frame
class BuildRandomTcTransferFrame:
    def __init__(self):
        self.builder = TcTransferFrameBuilder(False)

    def build(self,data,vcid,scid,vs):
        self.builder.set_virtual_channel_id(vcid)
        self.builder.set_spacecraft_id(scid)
        self.builder.set_frame_sequence_number(vs)
        self.builder.add_data(data)
        return self.builder.build()

Create sample directives/requests

In [12]:
import struct
import binascii

# Convert int32 array to binary
def int32array_to_binary(int32_array):
    binary_data = b''
    for value in int32_array:
        if (value<=2147483647):
            if endian == 'big':
                binary_data += struct.pack('>i', value)  # '>' for big-endian, 'i' for signed int32
            else:
                binary_data += struct.pack('<i', value)
        else:
            if endian == 'big':
                binary_data += struct.pack('>I', value)  
            else:
                binary_data += struct.pack('<I', value)    
    return binary_data

# Convert int8 array to binary
def int8array_to_bin(int8_array):
    binary_data = b''
    for value in int8_array:
        if endian == 'big':
            binary_data += struct.pack('>b', value)  # '>' for big-endian, 'b' for signed int8
        else:
            binary_data += struct.pack('<b', value)  # '<' for little-endian, 'b' for signed int8
    return binary_data

# Convert binary to int32 array
def bin_to_int32array(binary_data):
    padding_needed = len(binary_data) % 4
    if padding_needed:
        binary_data += b'\x00' * (4 - padding_needed)

    int32_array = []
    for i in range(0, len(binary_data), 4):
        if endian == 'big':
            int32_value = struct.unpack('>i', binary_data[i:i+4])[0]  
        else:
            int32_value = struct.unpack('<i', binary_data[i:i+4])[0]
        int32_array.append(int32_value)
    return int32_array

# Convert binary to int8 array
def bin_to_int8array(binary_data):
    int8_array = [int(byte) for byte in binary_data]
    return int8_array

# Convert binary to hexadecimal
def bin_to_hex(binary_data):
    hex_string = binascii.hexlify(binary_data).decode('utf-8')
    return hex_string

# Convert bits to bytes
def bits_to_bytes(bit_str_array):
    concatenated_bits = ''.join(bit_str_array)
    bytes_data = int(concatenated_bits, 2).to_bytes((len(concatenated_bits) + 7) // 8, byteorder='big')
    return bytes_data

# Convert int8 array to int32 array
def int8array_to_int32array(int8_array):
    padding = (4-len(int8_array)%4)%4
    int8_array.extend([0] * padding)
    int32_array = [int.from_bytes(int8_array[i:i+4], endian, signed=True) for i in range(0, len(int8_array), 4)]
    return int32_array

# Make a 223 byte data to mimick a TM transfer frame with the CLCW in the last 4 bytes (the other 219 bytes are just bit 1)
def make_tmframe(bit_str_array):
    bytes_data = bits_to_bytes(bit_str_array)
    additional_bytes = bytes([0xFF] * 219)
    final_bytes = additional_bytes + bytes_data
    return final_bytes

# Calculate 2s complement 32 bit checksum for the TCP message 
def calculate_checksum(packet):
    # Initialize the checksum
    checksum = 0
    # Sum up all the 32-bit words in the packet
    for word in packet:
        checksum += word
    # Take the one's complement
    checksum = ~checksum
    # Add 1 to the one's complement result
    checksum += 1
    # Take the result modulo 2^32
    checksum &= 0xFFFFFFFF
    return checksum

# Save the TCP message in a file 
def to_file(tcp_msg,name):
  print(tcp_msg)

  # Write the bytes to a binary file
  file_name = "fop_test_"+name+".txt"  # Name of the binary file

  with open(file_name, "wb") as binary_file:
      binary_file.write(tcp_msg)

# Optional: Run the cell for the required directive/request

Set V(S) Directive

In [13]:
#set vs directive
msg_size = 32
directive_type = 8
vcid = 60 # Virtual Channel ID (0 to 63) 
req_tag = 5 # Request Tag 
optional_parameter = 125 # New V(S) value (0 to 255)
postamble = -1234567890
msg_size_array = [
(msg_size >> 64) & 0xFFFFFFFF,
(msg_size >> 32) & 0xFFFFFFFF,
msg_size & 0xFFFFFFFF]
int32_packet = msg_size_array+[directive_type]+[vcid]+[req_tag]+[optional_parameter]+[postamble]
print("int32_packet=",int32_packet)
tcp_msg = int32array_to_binary(int32_packet)

to_file(tcp_msg,"set_vs")

int32_packet= [0, 0, 32, 8, 60, 5, 125, -1234567890]
b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00 \x00\x00\x00\x08\x00\x00\x00<\x00\x00\x00\x05\x00\x00\x00}\xb6i\xfd.'


Set Fop Sliding Window Width

In [14]:
#set fop sliding window directive
msg_size = 32
directive_type = 9
vcid = 60 # Virtual Channel ID (0 to 63) 
req_tag = 5 # Request Tag
optional_parameter = 5  # New FOP Sliding Window value (1 to 255)
postamble = -1234567890
msg_size_array = [
(msg_size >> 64) & 0xFFFFFFFF,
(msg_size >> 32) & 0xFFFFFFFF,
msg_size & 0xFFFFFFFF]
int32_packet = msg_size_array+[directive_type]+[vcid]+[req_tag]+[optional_parameter]+[postamble]
print("int32_packet=",int32_packet)
tcp_msg = int32array_to_binary(int32_packet)

to_file(tcp_msg,"setwindow")

int32_packet= [0, 0, 32, 9, 60, 5, 5, -1234567890]
b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00 \x00\x00\x00\t\x00\x00\x00<\x00\x00\x00\x05\x00\x00\x00\x05\xb6i\xfd.'


Set T1

In [15]:
#set t1 directive
msg_size = 32
directive_type = 10
vcid = 60 # Virtual Channel ID (0 to 63) 
req_tag = 5 # Request Tag
optional_parameter = 50 # New T1 directive value (timeout in seconds)
postamble = -1234567890
msg_size_array = [
(msg_size >> 64) & 0xFFFFFFFF,
(msg_size >> 32) & 0xFFFFFFFF,
msg_size & 0xFFFFFFFF]
int32_packet = msg_size_array+[directive_type]+[vcid]+[req_tag]+[optional_parameter]+[postamble]
print("int32_packet=",int32_packet)
tcp_msg = int32array_to_binary(int32_packet)

to_file(tcp_msg,"sett1")

int32_packet= [0, 0, 32, 10, 60, 5, 50, -1234567890]
b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00 \x00\x00\x00\n\x00\x00\x00<\x00\x00\x00\x05\x00\x00\x002\xb6i\xfd.'


Set Transmission Limit

In [34]:
#set transmission limit directive
msg_size = 32
directive_type = 11
vcid = 60 # Virtual Channel ID (0 to 63) 
req_tag = 5 # Request Tag
optional_parameter = 5 # New transmission limit value
postamble = -1234567890
msg_size_array = [
(msg_size >> 64) & 0xFFFFFFFF,
(msg_size >> 32) & 0xFFFFFFFF,
msg_size & 0xFFFFFFFF]
int32_packet = msg_size_array+[directive_type]+[vcid]+[req_tag]+[optional_parameter]+[postamble]
print("int32_packet=",int32_packet)
tcp_msg = int32array_to_binary(int32_packet)

to_file(tcp_msg,"settxlimit")

int32_packet= [0, 0, 32, 11, 60, 5, 5, -1234567890]
b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00 \x00\x00\x00\x0b\x00\x00\x00<\x00\x00\x00\x05\x00\x00\x00\x05\xb6i\xfd.'


Set Timeout Type

In [17]:
#set timeout type directive
msg_size = 32
directive_type = 12
vcid = 60 # Virtual Channel ID (0 to 63) 
req_tag = 5 # Request Tag
optional_parameter = 1 # New Timeout type (0 or 1)
postamble = -1234567890
msg_size_array = [
(msg_size >> 64) & 0xFFFFFFFF,
(msg_size >> 32) & 0xFFFFFFFF,
msg_size & 0xFFFFFFFF]
int32_packet = msg_size_array+[directive_type]+[vcid]+[req_tag]+[optional_parameter]+[postamble]
print("int32_packet=",int32_packet)
tcp_msg = int32array_to_binary(int32_packet)

to_file(tcp_msg,"settimeouttype")

int32_packet= [0, 0, 32, 12, 60, 5, 1, -1234567890]
b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00 \x00\x00\x00\x0c\x00\x00\x00<\x00\x00\x00\x05\x00\x00\x00\x01\xb6i\xfd.'


Init AD (Without CLCW) Directive

In [18]:
#initiate ad directive
msg_size = 32
directive_type = 2
vcid = 60 # Virtual Channel ID (0 to 63) 
req_tag = 5 # Request Tag
optional_parameter = 0
postamble = -1234567890
msg_size_array = [
(msg_size >> 64) & 0xFFFFFFFF,
(msg_size >> 32) & 0xFFFFFFFF,
msg_size & 0xFFFFFFFF]
int32_packet = msg_size_array+[directive_type]+[vcid]+[req_tag]+[optional_parameter]+[postamble]
print("int32_packet=",int32_packet)
tcp_msg = int32array_to_binary(int32_packet)

to_file(tcp_msg,"iniad")

int32_packet= [0, 0, 32, 2, 60, 5, 0, -1234567890]
b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00 \x00\x00\x00\x02\x00\x00\x00<\x00\x00\x00\x05\x00\x00\x00\x00\xb6i\xfd.'


Init AD (With CLCW) Directive

In [19]:
#initiate ad directive
msg_size = 32
directive_type = 3
vcid = 60 # Virtual Channel ID (0 to 63) 
req_tag = 5 # Request Tag
optional_parameter = 0
postamble = -1234567890
msg_size_array = [
(msg_size >> 64) & 0xFFFFFFFF,
(msg_size >> 32) & 0xFFFFFFFF,
msg_size & 0xFFFFFFFF]
int32_packet = msg_size_array+[directive_type]+[vcid]+[req_tag]+[optional_parameter]+[postamble]
print("int32_packet=",int32_packet)
tcp_msg = int32array_to_binary(int32_packet)

to_file(tcp_msg,"iniadwclcw")

int32_packet= [0, 0, 32, 3, 60, 5, 0, -1234567890]
b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00 \x00\x00\x00\x03\x00\x00\x00<\x00\x00\x00\x05\x00\x00\x00\x00\xb6i\xfd.'


Init AD (With Unlock) Directive

In [20]:
#initiate ad directive
msg_size = 32
directive_type = 4
vcid = 60 # Virtual Channel ID (0 to 63) 
req_tag = 5 # Request Tag
optional_parameter = 0
postamble = -1234567890
msg_size_array = [
(msg_size >> 64) & 0xFFFFFFFF,
(msg_size >> 32) & 0xFFFFFFFF,
msg_size & 0xFFFFFFFF]
int32_packet = msg_size_array+[directive_type]+[vcid]+[req_tag]+[optional_parameter]+[postamble]
print("int32_packet=",int32_packet)
tcp_msg = int32array_to_binary(int32_packet)

to_file(tcp_msg,"iniadwunlock")

int32_packet= [0, 0, 32, 4, 60, 5, 0, -1234567890]
b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00 \x00\x00\x00\x04\x00\x00\x00<\x00\x00\x00\x05\x00\x00\x00\x00\xb6i\xfd.'


Init AD (With set vr) Directive

In [36]:
#initiate ad directive
msg_size = 32
directive_type = 5
vcid = 60 # Virtual Channel ID (0 to 63) 
req_tag = 5 # Request Tag
optional_parameter = 120 # New V(R) value (0 to 255)
postamble = -1234567890
msg_size_array = [
(msg_size >> 64) & 0xFFFFFFFF,
(msg_size >> 32) & 0xFFFFFFFF,
msg_size & 0xFFFFFFFF]
int32_packet = msg_size_array+[directive_type]+[vcid]+[req_tag]+[optional_parameter]+[postamble]
print("int32_packet=",int32_packet)
tcp_msg = int32array_to_binary(int32_packet)

to_file(tcp_msg,"iniadwsetvr")

int32_packet= [0, 0, 32, 5, 60, 5, 120, -1234567890]
b'\x00\x00\x00\x00\x00\x00\x00\x00 \x00\x00\x00\x05\x00\x00\x00<\x00\x00\x00\x05\x00\x00\x00x\x00\x00\x00.\xfdi\xb6'


Terminate AD

In [22]:
#terminate ad directive
msg_size = 32
directive_type = 6
vcid = 60 # Virtual Channel ID (0 to 63) 
req_tag = 5 # Request Tag
optional_parameter = 0
postamble = -1234567890
msg_size_array = [
(msg_size >> 64) & 0xFFFFFFFF,
(msg_size >> 32) & 0xFFFFFFFF,
msg_size & 0xFFFFFFFF]
int32_packet = msg_size_array+[directive_type]+[vcid]+[req_tag]+[optional_parameter]+[postamble]
print("int32_packet=",int32_packet)
tcp_msg = int32array_to_binary(int32_packet)

to_file(tcp_msg,"terminatead")

int32_packet= [0, 0, 32, 6, 60, 5, 0, -1234567890]
b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00 \x00\x00\x00\x06\x00\x00\x00<\x00\x00\x00\x05\x00\x00\x00\x00\xb6i\xfd.'


Resume AD

In [23]:
#resume ad directive
msg_size = 32
directive_type = 7
vcid = 60 # Virtual Channel ID
req_tag = 5 # Request Tag
optional_parameter = 0
postamble = -1234567890
msg_size_array = [
(msg_size >> 64) & 0xFFFFFFFF,
(msg_size >> 32) & 0xFFFFFFFF,
msg_size & 0xFFFFFFFF]
int32_packet = msg_size_array+[directive_type]+[vcid]+[req_tag]+[optional_parameter]+[postamble]
print("int32_packet=",int32_packet)
tcp_msg = int32array_to_binary(int32_packet)

to_file(tcp_msg,"resumead")

int32_packet= [0, 0, 32, 7, 60, 5, 0, -1234567890]
b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00 \x00\x00\x00\x07\x00\x00\x00<\x00\x00\x00\x05\x00\x00\x00\x00\xb6i\xfd.'


TC Segment (AD)

In [37]:
#tc segment eg
operation_type = 20  #20 for AD 30 for BD
vcid = 60  # Virtual Channel ID (0 to 63) (byte 0)
encryption = 0 # no encryption: 0 encryption: 1 (byte 2)
req_tag = 5 # Request Tag
data_int8 = [0,0,0,33,0,0,0,44,0,0,55]  # TC segment (int8 array)
segment_length = len(data_int8)
data = int8array_to_int32array(data_int8) # Convert TC segment from int8 array to int32 array (padded with 0)
data_int8 = bin_to_int8array(int32array_to_binary(data)) # padded TC segment (int8 array)
data_hex = bin_to_hex(int32array_to_binary(data)) # padded TC segment (hexadecimal)
msg_size = len(data)*4 + 9*4 
print("tc segment:",data_hex)
postamble = -1234567890
msg_size_array = [
(msg_size >> 64) & 0xFFFFFFFF,
(msg_size >> 32) & 0xFFFFFFFF,
msg_size & 0xFFFFFFFF]

value1 = vcid.to_bytes(1, byteorder='big', signed=True)
value2 = encryption.to_bytes(1, byteorder='big', signed=True)
result_int32 = 0
result_int32 |= value1[0] # Place value1 in byte 0
result_int32 |= value2[0] << 16 # Place value2 in byte 2

int32_packet = msg_size_array+[operation_type]+[result_int32]+[req_tag]+[segment_length]+data+[postamble]
checksum = calculate_checksum(int32_packet)
print("checksum=",checksum)
tc_transfer_req = msg_size_array+[operation_type]+[result_int32]+[req_tag]+[segment_length]+data+[checksum]+[postamble]
print("segment_transfer_req=",tc_transfer_req)
tcp_msg = int32array_to_binary(tc_transfer_req)
#end of tc segment eg

to_file(tcp_msg,"segmentad")

tc segment: 000000210000002c00003700
checksum= 4234084930
segment_transfer_req= [0, 0, 48, 20, 60, 5, 11, 553648128, 738197504, 3604480, 4234084930, -1234567890]
b'\x00\x00\x00\x00\x00\x00\x00\x000\x00\x00\x00\x14\x00\x00\x00<\x00\x00\x00\x05\x00\x00\x00\x0b\x00\x00\x00\x00\x00\x00!\x00\x00\x00,\x00\x007\x00B\x02_\xfc.\xfdi\xb6'


TC Segment (BD)

In [38]:
#tc segment eg
operation_type = 30  #20 for AD 30 for BD
vcid = 60  # Virtual Channel ID (0 to 63) (byte 0)
encryption = 0 # no encryption: 0 encryption: 1 (byte 2)
req_tag = 5 # Request Tag
data_int8 = [0,0,0,33,0,0,0,44,0,0,55] * 50 # TC segment (max 1019 bytes/ 1017 bytes (if 2 bytes fecf present)) (int8 array) 
segment_length = len(data_int8)
data = int8array_to_int32array(data_int8) # Convert TC segment from int8 array to int32 array (padded with 0)
data_int8 = bin_to_int8array(int32array_to_binary(data)) # padded TC segment (int8 array)
data_hex = bin_to_hex(int32array_to_binary(data)) # padded TC segment (hexadecimal)
msg_size = len(data)*4 + 9*4 
print("tc segment:",data_hex)
postamble = -1234567890
msg_size_array = [
(msg_size >> 64) & 0xFFFFFFFF,
(msg_size >> 32) & 0xFFFFFFFF,
msg_size & 0xFFFFFFFF]

byte0 = vcid.to_bytes(1, byteorder='big', signed=True)
byte2 = encryption.to_bytes(1, byteorder='big', signed=True)
result_int32 = 0
result_int32 |= byte0[0]  # Place value1 in byte 0 
result_int32 |= byte2[0] << 16 # Place value2 in byte 2 

int32_packet = msg_size_array+[operation_type]+[result_int32]+[req_tag]+[segment_length]+data+[postamble]
checksum = calculate_checksum(int32_packet)
print("checksum=",checksum)
tc_transfer_req = msg_size_array+[operation_type]+[result_int32]+[req_tag]+[segment_length]+data+[checksum]+[postamble]
print("segment_transfer_req=",tc_transfer_req)
tcp_msg = int32array_to_binary(tc_transfer_req)
#end of tc segment eg

to_file(tcp_msg,"segmentbd")

tc segment: 000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c0000

TC Transfer Frame

In [26]:
#tc transfer frame eg
operation_type = 50
codeblock_length = 0
vcid = 60 # Virtual Channel ID (0 to 63) (byte 0)
req_tag = 5 # Request Tag
scid = 80 # Spacecraft ID
vs = 1 # Frame Sequence Number V(S) (0 to 255)
tc_data_int8 = [0,0,0,33,0,0,0,44,0,0,55] # TC transfer frame data field (int8 array)
frame_length = len(tc_data_int8) + 5 # Data field length + header length (5 bytes)
tc_data = int8array_to_bin(tc_data_int8) # TC transfer frame data field (binary)
print("tc_data",tc_data)
data_bin = BuildRandomTcTransferFrame().build(tc_data,vcid,scid,vs).frame # TC transfer frame (binary)
data = bin_to_int32array(data_bin) # Convert TC transfer frame from binary to int32 array (padded with 0)
data_int8 = bin_to_int8array(data_bin) # padded TC transfer frame (int8 array)
data_hex = bin_to_hex(data_bin) # padded TC transfer frame (hexadecimal)
msg_size = len(data)*4 + 9*4 
print("tcframe:",data_hex)
postamble = -1234567890
msg_size_array = [
(msg_size >> 64) & 0xFFFFFFFF,
(msg_size >> 32) & 0xFFFFFFFF,
msg_size & 0xFFFFFFFF]
int32_packet = msg_size_array+[operation_type]+[codeblock_length]+[req_tag]+[frame_length]+data+[postamble]
checksum = calculate_checksum(int32_packet)
print("checksum=",checksum)
tc_transfer_req = msg_size_array+[operation_type]+[codeblock_length]+[req_tag]+[frame_length]+data+[checksum]+[postamble]
print("tc_transfer_req=",tc_transfer_req)
tcp_msg = int32array_to_binary(tc_transfer_req)
#end of tc transfer frame eg

to_file(tcp_msg,"tcframe")

tc_data b'\x00\x00\x00!\x00\x00\x00,\x00\x007'
tcframe: 0050f00f01000000210000002c000037
checksum= 4215607825
tc_transfer_req= [0, 0, 52, 50, 0, 5, 16, 5304335, 16777216, 553648128, 738197559, 4215607825, -1234567890]
b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x004\x00\x00\x002\x00\x00\x00\x00\x00\x00\x00\x05\x00\x00\x00\x10\x00P\xf0\x0f\x01\x00\x00\x00!\x00\x00\x00,\x00\x007\xfbE\x12\x11\xb6i\xfd.'


TC Packet

In [27]:
#tc packet eg 
operation_type = 60  #60 for AD 70 for BD
vcid = 60 # Virtual Channel ID (0 to 63) (byte 0)
mapid = 15 # MAP ID (0 to 63) (byte 1)
encryption = 0 # no encryption: 0 encryption: 1 (byte 2)
req_tag = 5 # Request Tag
data_int8 = [0,0,0,33,0,0,0,44,0,0,55] * 100 # TC packet (max 4096 bytes) (int8 array) 
packet_length = len(data_int8)
data = int8array_to_int32array(data_int8) # Convert TC packet from int8 array to int32 array (padded with 0)
data_int8 = bin_to_int8array(int32array_to_binary(data)) # padded TC packet (int8 array)
data_hex = bin_to_hex(int32array_to_binary(data)) # padded TC packet (hexadecimal)
msg_size = len(data)*4 + 9*4 
print("tc packet:",data_hex)
postamble = -1234567890
msg_size_array = [
(msg_size >> 64) & 0xFFFFFFFF,
(msg_size >> 32) & 0xFFFFFFFF,
msg_size & 0xFFFFFFFF]

byte0 = vcid.to_bytes(1, byteorder='big', signed=True)
byte1 = mapid.to_bytes(1, byteorder='big', signed=True)
byte2 = encryption.to_bytes(1, byteorder='big', signed=True)
result_int32 = 0
result_int32 |= byte0[0] # Place byte0 in byte 0
result_int32 |= byte1[0] << 8 # Place byte1 in byte 1
result_int32 |= byte2[0] << 16 # Place byte2 in byte 2 

int32_packet = msg_size_array+[operation_type]+[result_int32]+[req_tag]+[packet_length]+data+[postamble]
checksum = calculate_checksum(int32_packet)
print("checksum=",checksum)
tc_transfer_req = msg_size_array+[operation_type]+[result_int32]+[req_tag]+[packet_length]+data+[checksum]+[postamble]
print("packet_transfer_req=",tc_transfer_req)
tcp_msg = int32array_to_binary(tc_transfer_req)
#end of tc packet eg

to_file(tcp_msg,"packet")

tc packet: 000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c000037000000210000002c00003

CLCW

In [33]:
#clcw
ctr_word = '0'
clcw_version = '00'
status_field = '000'
cop_in_effect = '01' #COP-1
vcid = '111100' #60
rsvd_spare = '00'
no_rf_flag = '0' #0:available 1:not available (for all VCs)
no_bit_lock_flag = '0' #0:bit lock 1:no bit lock
lock_out_flag = '0' #0:no lockout 1:lockout(all further type A rejected)
wait_flag = '0' #0:don't wait 1:wait(all further type A rejected)
retransmit_flag = '0' #0:do retransmit needed 1:need retransmit
farm_b_counter = '00' #count no of type B frame accepted in bypass mode
rsvd_spare2 = '0'
report_value = '00000001' #N(R) #120: 1111000
clcw = [ctr_word,clcw_version,status_field,cop_in_effect,vcid,rsvd_spare,no_rf_flag,no_bit_lock_flag,lock_out_flag,wait_flag,retransmit_flag,
        farm_b_counter,rsvd_spare2,report_value]
tm_frame = make_tmframe(clcw) # Make a 223 byte data to mimick a TM transfer frame with the CLCW in the last 4 bytes (the other 219 bytes are just bit 1)
tcp_msg = tm_frame
# end of clcw

to_file(tcp_msg,"clcw")

b'\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\x00\xf8\x00y'
